In [1]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
from tenacity import retry, stop_after_attempt, wait_exponential, RetryError

URL_PAGE = "https://www.gov.uk/government/statistics/oil-and-oil-products-section-3-energy-trends"
TARGET_MATCH_PHRASE = "Supply and use of crude oil, natural gas liquids and feedstocks"
LAST_LINK_FILE = 'last_link.txt'

In [2]:
def extract_link_from_jsonld(soup):
    script_tags = soup.find_all("script", type="application/ld+json")
    for tag in script_tags:
        try:
            data = json.loads(tag.string)
        except json.JSONDecodeError:
            continue

        items = data if isinstance(data, list) else [data]
        for item in items:
            if item.get('@type') == 'BreadcrumbList':
                continue

            distributions = item.get("distribution", [])
            if isinstance(distributions, dict):
                distributions = [distributions]
            for d in distributions:
                if TARGET_MATCH_PHRASE in d.get("name", ""):
                    return d.get("contentUrl")
    return None        

In [3]:
def load_last_link(filepath=LAST_LINK_FILE):
    if os.path.exists(filepath):
        with open(filepath, 'r', encoding='utf-8') as f:
            return f.read().strip()
    return None

In [4]:
def save_last_link(link, filepath=LAST_LINK_FILE):
    with open(filepath, 'w', encoding='utf-8') as f:
        f.write(link)

In [5]:
@retry(stop=stop_after_attempt(3), wait=wait_exponential(multiplier=1, min=4, max=10))
def download_file_with_retry(url, filename="energy_data.xlsx"):
    response = requests.get(url)
    response.raise_for_status()
    
    with open(filename, "wb") as file:
        file.write(response.content)

    print(f"File downloaded: {filename}")
    return filename

In [7]:
def main():
    response = requests.get(URL_PAGE)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, 'lxml')

    new_link = extract_link_from_jsonld(soup)

    if not new_link:
        print("Could not extract the spreadsheet link.")
        return
    if LAST_LINK_FILE == new_link:
        print("Link unchanged. Exiting early.")
        return
    if LAST_LINK_FILE != new_link:
        print(f"New spreadsheet link found {new_link}")
        try:
            downloaded_file = download_file_with_retry(new_link)
        except RetryError as e:
            print("Download failed after all retries:", e)

main()

New spreadsheet link found https://assets.publishing.service.gov.uk/media/6762b1d5cdb5e64b69e3073b/ET_3.1_DEC_24.xlsx
File downloaded: energy_data.xlsx


In [9]:
df = pd.read_excel("energy_data.xlsx", sheet_name="Quarter")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23 entries, 0 to 22
Columns: 104 entries, Table 3.1 Supply and use of crude oil, natural gas liquids and feedstocks, quarterly data (thousand tonnes) [note 1] to Unnamed: 103
dtypes: object(104)
memory usage: 18.8+ KB
